# Using CrewAI with Ollama for Amazon and DuckDuckGo Searches

This notebook demonstrates how to:
- Install the necessary libraries
- Set up and use CrewAI's tools for performing Amazon and web searches
- Configure and use an Ollama language model with CrewAI for a simple shopping assistant task.

In [ ]:
# Step 1: Install CrewAI and other necessary libraries
!pip install crewai
!pip install langchain_community

### Step 2: Import Libraries and Initialize Ollama Server
We will import the necessary classes and start the Ollama server in the background if it isn't already running.

In [ ]:
import crewai
from langchain_community.tools import DuckDuckGoSearchRun
from crewai_tools import tool, WebsiteSearchTool
from crewai import Agent, LLM

# Start Ollama server in the background
!ollama serve &>/dev/null&

### Step 3: Configure the LLM Model
We will configure the LLM model using Ollama's LLaVA with a specified temperature and token limit.

In [ ]:
# Configure the LLM model
llm = LLM(
    model="ollama/llava:13b-v1.6",
    temperature=0.7,
    max_tokens=1000
)
print("LLM configured.")

### Step 4: Define Search Tools
Define tools for performing Amazon and DuckDuckGo searches using CrewAI. These tools will be used by the agent during its task.

In [ ]:
@tool("Search Amazon")
def search_amazon(q: str) -> str:
    """Search Amazon"""
    return DuckDuckGoSearchRun().run(f"site:https://amazon.com {q}")

@tool("DuckDuckGoSearch")
def search(search_query: str):
    """Search the web for information on a given topic"""
    return DuckDuckGoSearchRun().run(search_query)

print("Search tools defined.")

### Step 5: Define Callback Function
Define a callback function to handle the task output once completed.

In [ ]:
def callback_function(output):
    print(f"Task completed: {output.raw_output}")

### Step 6: Define the Prompt and Configure the Agent
We will set up the agent with a specific role and goal to search for information about a good laptop under $1000 using the tools and the LLM model configured earlier.

In [ ]:
prompt = '''Find info on a good laptop under $1000'''

agent_amazon_shopper = crewai.Agent(
    role="Amazon Shopper",
    goal="Find info about a good laptop under $1000",
    backstory="As a savvy shopper, you need to find info on a good laptop under $1000",
    tools=[search],
    llm=llm,
    allow_delegation=False, verbose=True
)

print("Agent configured.")

### Step 7: Create Task and Run the Crew
Define a task for the agent and initiate the crew to start the task execution.

In [ ]:
task_shop = crewai.Task(description=prompt,
                   agent=agent_amazon_shopper,
                   expected_output='''info on laptop''')

crew = crewai.Crew(agents=[agent_amazon_shopper], tasks=[task_shop], verbose=True)
res = crew.kickoff()
print(res)